# Pandas (1)

In [ ]:
import pandas as pd

Els objectes principals de Pandas són els `DataFrames` --- venen a ser taules de dades amb files i columnes.

Agafem les dades de les línies de bus que ja coneixem 

In [ ]:
df = pd.read_csv("data/ctabus.csv")
df

## Veure les dades

Mètodes per explorar un `DataFrame`:

- `head(n)`-  Mostra les n primeres columnes
- `tail(n)`-  Mostra les n últimes columnes
- `info` - Ens dius quantes entrades no buides hi ha, i el tipus de variable de cada columna
- `describe` - Mostra estadístiques bàsiques de les columnes numèriques

In [ ]:
df.head()

In [ ]:
# Dtype object vol dir string per Pandas df.info()

In [ ]:
df.describe()

Com que Pandas fa servir NumPy, podem fer servir els mètodes i atributs de NumPy

A més, anirem veient altres atributs interessants, com `columns`

In [ ]:
(
    df.shape,
    df.size,
    df.ndim,
    len(df),
)

In [ ]:
df.columns

## Indexar

S'indexa una sola columna igual que un diccionari. El resultat és una `Series` de Pandas.

In [ ]:
# Agafem 5 files per no imprimir tant
df = df[75:80]

In [ ]:
df["route"]

També es pot indexar com si fós un atribut (si el nom no té espais)

In [ ]:
df.route

Si volem indexar diverses columnes, ens cal una llista

In [ ]:
cols = ["route", "rides"]
df[cols]

In [ ]:
# O el que és el mateix
df[["route", "rides"]]

Per **indexar files** farem servir `.loc`

In [ ]:
# Una sola fila també retorna una `Series`
df.loc[75]

In [ ]:
df.loc[[75, 76, 78]]

In [ ]:
# Podem fer servir rangs (però cal tenir compte!). Què hi veieu?
df.loc[:77]

Els índexs no tenen per què ser números, per això els rangs amb `loc` són inclusius.

Per exemple, podem fer servir una columna com a índex amb `set_index`

In [ ]:
route_df = df.set_index("route")
route_df

In [ ]:
route_df.loc["49B"]

In [ ]:
route_df.loc["52A":"54B"]

Per 'recuperar' l'índex com a columna fem `.reset_index()`

In [ ]:
route_df.reset_index()

Fixeu-vos que les operacions no canvien l'objecte si no fem un =

In [ ]:
route_df

Si en canvi volem indexar numèricament, sense fixar-nos en el nom de l'índex o les columnes, farem servir `iloc`

In [ ]:
df.iloc[0]

In [ ]:
df.iloc[:3]

In [ ]:
df.iloc[:3, 1:]

**Resum**


+ `[]` per selccionar columnes 
+ `.loc[noms_files, noms_columnes]` per indexar per nom
+ `.iloc[poscio_files, posicio_columnes]` per indexar per posició

### Exercici 0

Posa-ho tot junt. Selecciona les primeres 500 files senars (de l'índex 1 al 999) i les columnes "route", "date" i "rides"

In [ ]:
df = pd.read_csv("data/ctabus.csv")

## Series

Les columnes d'un DataFrame són `Series` i tenen mètodes específics.

- `unique()` - elements únics (el que fèiem amb un set)
- `nunique()` - nombre d'elements únics
- `value_counts()` - recompte d'elements (el que fèiem amb un Counter)

In [ ]:
df.route.nunique()

In [ ]:
df.daytype.unique()

De fet, `np.unique` és també una funció de NumPy

In [ ]:
import numpy as np

np.unique(np.array([1, 1, 2, 2, 3, 3, 4, 5]))

I `.nunique()` serveix també per tot un `DataFrame` senser

In [ ]:
df.nunique()

## Modificar les dades

In [ ]:
df = pd.read_csv("data/ctabus.csv")

### Crear i modificar 

És fàcil crear columnes diferents a partir de les ja existents

In [ ]:
df["rides_milers"] = df.rides / 1000
df["ruta_dia"] = df["route"] + "_" + df["date"]
df.head()

Per fer servir funcions relacionades amb strings, hem de fer primer `.str`

In [ ]:
df.date.str.replace("20", "").head()

In [ ]:
df.date.str.split("/").head()

Si volem canviar un valor a la mateixa columna la reassignem

In [ ]:
df.date = df.date.str.replace("20", "").str.split("/")
df.head()

És fàcil modificar una columna sencera

In [ ]:
df.rides = 3
df.head()

Però, un 'error' molt típic a Pandas apareix quan volem modificar certes posicions.

Llegeix-lo

In [ ]:
df.rides.loc[:5] = 3

In [ ]:
df.loc[:5, "rides"] = 5
df.head(10)

### Esborrar

Per esborrar files o columnes es fa servir `.drop()`

In [ ]:
df.drop(range(5))  # axis=0 per defecte (files)

In [ ]:
df.drop("daytype", axis=1).head()  # per borrar columnes cal axis=1

In [ ]:
df.drop(["daytype", "route"], axis=1).head()

### Canviar nom columnes

`df.rename(columns={"columna vella": "columna nova"})`

In [ ]:
df.rename(columns={"route": "ruta", "date": "data", "daytype": "tipus de dia", "rides": "viatgers"}).head()

## Ordenar i agrupar

### Ordenar

Un DataFrame s'ordena per una (o vàries) columnes amb `sort_values`. Per defecte l'ordre és ascendent, però el podem girar amb `ascending=False`

In [ ]:
df.sort_values(by="rides")

In [ ]:
df.sort_values(by="rides", ascending=False)

In [ ]:
df.sort_values(by=["rides", "daytype"])

In [ ]:
df.sort_values(by=["rides", "daytype"], ascending=False)

In [ ]:
# es pot ordenar per vàries columnes amb diferent ordre
df.sort_values(by=["rides", "daytype"], ascending=[True, False])

Les `Series` s'ordenen igual, però no cal passar cap argument, ja que no tenen columnes

In [ ]:
df.route.sort_values()

### Agrupar

`groupby` agrupa per columnes. Si el fem servir sol només retorna un objecte de pandas, però sense cap informació.
Cal aplicar-hi una operació per tenir algun resultat. Les operacions típiques que ens poden interessar són: 

`count()`, `sum()`, `mean()`, `median()` i `std()`.

Si no volem aplicar aquesta operació a totes les columnes haurem de seleccionar les que volem **abans** de l'operació, per ser més eficients.

In [ ]:
df.groupby("route")["rides"].median()

Si volem vàries operacions alhora hem de fer servir `.agg()`

In [ ]:
df.groupby("route")["rides"].agg(["count", "mean", "std"])

## Exercici 1

1. Canvia el nom de les dues últimes columnes perquè no tinguin espais
2. Crea una columna "ratio" que sigui el temps del segons experiment dividit pel temps del primer.
3. Esborra la columna timestamp
4. Ordena el df per gènere i edat (en aquest ordre), el dos en ordre ascendent.
5. Agrupa per gènere i mostra la mitja i la desviació estàndard dels dos temps i del ràtio.

In [ ]:
df = pd.read_csv("data/efecte_stroop.csv")
df.head()

### Solució

In [ ]:
df = (
    pd.read_csv("data/efecte_stroop.csv")
    .rename(columns={"Temps experiment 1": "experiment_1", "Temps experiment 2": "experiment_2"})
    .drop("Timestamp", axis=1)
    .sort_values(by=["Gènere", "Edat"])
)

df["ratio"] = df["experiment_2"] / df["experiment_1"]
df

In [ ]:
df.groupby("Gènere")[["experiment_1", "experiment_2", "ratio"]].agg(["mean", "std"])

## Llegir i escriure fitxers

Podem llegir dades d'un CSV, Excel, SQL, JSON ...

Les funcions són bastant similars: `pd.read_csv`, `pd.read_excel`...

Segons el format que vulguem llegir i el nostre sistema operatiu és possible que calgui instalar algun altre paquet.

Ens centrarem en fitxers `csv`.
Tot i que el nom ve de "comma separated values", el separador pot ser diferent d'una coma. Canviant l'argument `sep` es poden llegir fitxers amb diferents separadors, com tabuladors "\t" (sovint amb fitxers '.tsv').

In [ ]:
# Hi ha moltes opcions
pd.read_csv?

Per escriure fitxer, igualment hi ha una funció per cada tipus de dades: `pd.to_csv`, `pd.to_excel`, `to.to_latex`...

## Crear un `DataFrame` 

També podem crear un `DataFrame`a partir d'un diccionari, llista, ser, array ...

### `DataFrame` des d'un diccionari

#### Cada clau és una columna

In [ ]:
dict_estudiants = {"nom": ["Maria", "Berta", "Edu", "Arnau"], "edat": [20, 21, 19, 23], "nota": [8.5, 7.7, 9.4, 5.9]}
df_estudiants = pd.DataFrame(dict_estudiants)
df_estudiants

In [ ]:
df = pd.DataFrame({"X": [78, 85, 96, 80, 86], "Y": [84, 94, 89, 83, 86], "Z": [86, 97, 96, 72, 83]})
df

Si ens volem quedar només amb unes quantes columnes, les podem seleccionar amb `columns=['col1', 'col2']`

In [ ]:
df_estudiants = pd.DataFrame(dict_estudiants, columns=["nom", "nota"])
df_estudiants

Per defecte, es crea una primera columna, _l'índex_, amb nombres naturals

In [ ]:
df_estudiants.index

El podem modificar quan creem el `DataFrame` amb `index=['index1','index2']`

In [ ]:
df_estudiants = pd.DataFrame(dict_estudiants, index=["est1", "est2", "est3", "est4"])
df_estudiants

In [ ]:
df_estudiants.index

#### Cada clau és una fila

In [ ]:
dict_estudiants = {
    "Maria": 8.5,
    "Berta": 7.7,
    "Edu": 9.4,
    "Arnau": 5.9,
}
df_estudiants = pd.DataFrame(dict_estudiants.items())
df_estudiants

Ara Pandas no sap quin nom posar a les columnes, així que hi ha posat números. Posa-hi noms

In [ ]:
student_dict = {"Grade A": ["Joe", "Harry"], "Grade B": ["Nat"]}
print(student_dict)

student_df = pd.DataFrame.from_dict(student_dict, "index").stack().reset_index(level=0)
student_df

### `DataFrame` des d'una llista

In [ ]:
llista_fruites = ["Poma", "Mango", "Meló", "Cireres"]
df_fruites = pd.DataFrame(llista_fruites, columns=["Fruites"])
df_fruites

In [ ]:
llista_fruites = [["Poma", 1], ["Mango", 1.5], ["Meló", 1.2], ["Cireres", 2.3]]
df_fruites = pd.DataFrame(llista_fruites, columns=["Fruites", "Preu"])
df_fruites

En realitat, quan hem creat un `DataFrame` a partir dels items d'un diccionari, l'estructurura era la mateixa 

In [ ]:
dict_estudiants.items()

Si partim de dues o més llistes, les podem posar amb el format adequat amb zip

In [ ]:
llista_fruites = ["Poma", "Mango", "Meló", "Cireres"]
llista_preus = [1, 1.5, 1.2, 2.3]
df_fruites = pd.DataFrame(list(zip(llista_fruites, llista_preus)), columns=["Fruites", "Preu"])
df_fruites

## Concatenar operacions

Pandas permet fer línies molt llargues concatenant operacions, en comptes de reassignar el resultat cada vegada.

Perquè sigui més llegible se sol posar cada operació en una nova línia dins de parèntesis ()

In [ ]:
df = pd.read_csv("data/ctabus.csv")
df = df.rename(columns={"route": "ruta", "date": "data", "daytype": "tipus de dia", "rides": "viatgers"})
df = df.drop("tipus de dia", axis=1)
df.head()

In [ ]:
df = (
    pd.read_csv("data/ctabus.csv")
    .rename(columns={"route": "ruta", "date": "data", "daytype": "tipus de dia", "rides": "viatgers"})
    .drop("tipus de dia", axis=1)
)
df.head()

## Exercici 2

In [ ]:
df = pd.read_csv("data/ctabus.csv")

Refés les següent preguntes de l'exercici final del primer notebook, ara amb Pandas:

1. Quants dies hi ha de cada tipus?
2. Quants viatgers totals hi ha a les dades per cada línia?
3. Quines són les 5 línies amb més viatgers totals?

4. Quantes persones van anar a l'autobús número 22 el 2 de febrer de 2011? Fes una funció per qualsevol altre dia i ruta.
5. *Quines 5 línies han incrementat més el seu volum de viatgers entre el 2001 i el 2011?

### Solució

In [ ]:
# 1. Nombre de dies de cada tipus
df.daytype.value_counts()

In [ ]:
# 2.
viatges_totals = df.groupby("route")["rides"].sum()
viatges_totals

In [ ]:
# 3. Línies amb més viatges
viatges_totals.sort_values()[-5:]

In [ ]:
# 4. Línia i dia concrets
df[(df.route == "22") & (df.date == "02/02/2011")].rides

L'última pregunta és una mica difícil.

Si intentem fer el mateix que amb un Counter, al restar surten NaNs, ja que hi ha algunes línies que només estan a un dels dos anys.

Aquest és el cas de la línia "15", que hauria de ser la primera, però no ens surt

In [ ]:
viatges_2001 = df[df.date.str.contains("2001")].groupby("route")["rides"].sum()
viatges_2011 = df[df.date.str.contains("2011")].groupby("route")["rides"].sum()

diff = viatges_2011 - viatges_2001
diff.sort_values(ascending=False)

El que hem de fer és crear un segon DataFrame amb el nombre de viatgers per any com a columes.
Hi ha vàries maneres de fer-ho. Una és amb `pd.concat`.

Un cop ho tenim junt hi seguirà havent NaNs, però ara els podem convertir a 0 i després fer la resta

In [ ]:
df_anys = pd.concat((viatges_2001, viatges_2011), axis=1, keys=["2001", "2011"])
df_anys[df_anys.isna()] = 0
df_anys["diff"] = df_anys["2011"] - df_anys["2001"]
df_anys.sort_values(by="diff")